In [ ]:
import pandas as pd
import numpy as np

from sklearn.preprocessing import StandardScaler

In [ ]:
SEQ_LEN = 30
INTERVAL = 5
assert 0 < INTERVAL < 60 

In [ ]:
TRAIN = 0.7
VAL = 0.15
TEST = 0.15
assert TRAIN + VAL + TEST == 1

#### Load data

In [ ]:
df_og = pd.read_csv('../sample_data/avg_from_2020_10_01_00_00_00_to_2020_10_01_23_59_00')
df_og['czas'] = pd.to_datetime(df_og['czas'])
df_og.set_index('czas', inplace=True)

temp_zuz = pd.read_csv('../sample_data/temp_zuz_fixed.csv', delimiter=',')
temp_zuz['czas_converted'] = pd.to_datetime(temp_zuz['czas_converted'])
temp_zuz.set_index('czas_converted', inplace=True)

In [ ]:
temp_zuz.head()

In [ ]:
df_og.head()

### Proste featury wybieram albo tworze agregując


In [ ]:
df = pd.DataFrame(index=df_og.index)

In [ ]:
# srednia po 'WODY POWROTNE KOLEKTORÓW [°C]'
TIR = df_og.columns[df_og.columns.str.contains('tir')].values
df['TIR'] = df_og[TIR].mean(axis='columns')

In [ ]:
# srednia po 'TEMP POD 2 WARSTWĄ WYMURÓWKI [°C]'
TIX1 = df_og.columns[df_og.columns.str.contains('001tix')].values
df['TIX1'] = df_og[TIX1].mean(axis='columns')

In [ ]:
# wrzucam o prażonkach i prob
PR = df_og.columns[df_og.columns.str.contains('pr')].values
df[PR] = df_og[PR]

### Features to pct change

In [ ]:
df = df.pct_change()

In [ ]:
df

### Dołaczam temp_zuz i wywalam 

In [ ]:
df.join(temp_zuz['temp_zuz'])

### Rozdzielam datasety

In [ ]:
df

In [ ]:
df_train_X = iloc

### Scale

In [ ]:
scaler = StandardScaler().fit(df.values)

scaler 

scaler.mean_, scaler.scale_